# 40 Eridani A Mission Analysis
This notebook loads UQ and perf artifacts generated by the pipeline and summarizes feasibility and energy variability.

In [ ]:
# Imports
import json
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
# Load artifacts (paths relative to repo root)
uq_summary_path = Path('uq_summary.json')
uq_records_path = Path('uq_records.jsonl')
perf_summary_path = Path('perf_aggregate.json')

uq_summary = json.loads(uq_summary_path.read_text()) if uq_summary_path.exists() else {}
records = []
if uq_records_path.exists():
    with open(uq_records_path) as f:
        for line in f:
            line = line.strip()
            if line:
                records.append(json.loads(line))
perf_summary = json.loads(perf_summary_path.read_text()) if perf_summary_path.exists() else {}

len(records), uq_summary.get('feasible_fraction'), uq_summary.get('energy_cv')

In [ ]:
# Compute basic metrics and plot energy distribution
import math
energies = [r['planned_energy'] for r in records if 'planned_energy' in r]
if energies:
    mean_e = sum(energies)/len(energies)
    var_e = sum((e-mean_e)**2 for e in energies)/max(1, len(energies)-1)
    std_e = math.sqrt(var_e)
    cv_e = std_e/mean_e if mean_e>0 else float('nan')
    print({'energy_mean': mean_e, 'energy_std': std_e, 'energy_cv': cv_e, 'feasible_fraction': uq_summary.get('feasible_fraction')})
    plt.figure(figsize=(6,4))
    plt.hist(energies, bins=15, color='#4C78A8', edgecolor='white')
    plt.title('Planned Energy Distribution')
    plt.xlabel('Energy (J)')
    plt.ylabel('Count')
    plt.tight_layout()
    plt.savefig('40eridani_energy.png', dpi=150)
else:
    print('No records found.')

In [ ]:
# Feasibility over samples (binary feasible field)
import numpy as np
feas = [1 if r.get('feasible') else 0 for r in records]
if feas:
    n = len(feas)
    win = max(1, n // 10)
    roll = np.convolve(feas, np.ones(win)/win, mode='same')
    plt.figure(figsize=(6,4))
    plt.plot(roll, color='#F58518')
    plt.ylim(0,1)
    plt.title('Feasible Fraction (rolling)')
    plt.xlabel('Sample Index')
    plt.ylabel('Feasible Fraction')
    plt.tight_layout()
    plt.savefig('40eridani_feasibility.png', dpi=150)
else:
    print('No feasibility data.')

In [ ]:
# UQ with varied distance profile (5x5e13, 10x7e13, 5x9e13) totaling 1.4e15 m
import json, os, subprocess, sys, textwrap
from pathlib import Path
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np

repo_root = Path.cwd().resolve().parents[0] if (Path.cwd().name == 'notebooks') else Path.cwd().resolve()
data_path = repo_root / 'data' / 'dist_profile_40eridani_varied.csv'
assert data_path.exists(), f"Missing varied profile: {data_path}"

# Run the UQ runner for the varied profile
summary = repo_root / 'uq_summary_varied.json'
records = repo_root / 'uq_records_varied.jsonl'
cmd = [sys.executable, '-m', 'src.uq_validation.impulse_uq_runner', '--samples', '100', '--seed', '123', '--out', str(summary), '--jsonl-out', str(records), '--dist-profile', str(data_path)]
print('Running:', ' '.join(cmd))
subprocess.run(cmd, check=True, cwd=str(repo_root))

# Load outputs
summ = json.loads(summary.read_text()) if summary.exists() else {}
recs = [json.loads(l) for l in records.read_text().splitlines() if l.strip()] if records.exists() else []

energy_vals = [r.get('planned_energy', 0.0) for r in recs]
energy_mean = float(np.mean(energy_vals)) if energy_vals else float('nan')
energy_std = float(np.std(energy_vals)) if energy_vals else float('nan')
energy_cv = float(energy_std / energy_mean) if energy_vals and energy_mean else float('nan')
feasible_fraction = float(np.mean([1 if r.get('feasible') else 0 for r in recs])) if recs else float('nan')
print('Varied Profile Metrics:', {'energy_mean': energy_mean, 'energy_std': energy_std, 'energy_cv': energy_cv, 'feasible_fraction': feasible_fraction})

# Plots
plt.figure(figsize=(6,4))
if energy_vals:
    plt.hist(energy_vals, bins=20, color='#54A24B', edgecolor='white')
plt.title('Planned Energy Distribution (Varied Profile)')
plt.xlabel('Energy (J)')
plt.ylabel('Count')
plt.tight_layout()
plt.savefig(repo_root / '40eridani_energy_varied.png', dpi=150)

feas_arr = [1 if r.get('feasible') else 0 for r in recs]
if feas_arr:
    n = len(feas_arr)
    win = max(1, n//10)
    roll = np.convolve(feas_arr, np.ones(win)/win, mode='same')
    plt.figure(figsize=(6,4))
    plt.plot(roll, color='#E45756')
    plt.ylim(0,1)
    plt.title('Feasible Fraction (rolling, Varied Profile)')
    plt.xlabel('Sample Index')
    plt.ylabel('Feasible Fraction')
    plt.tight_layout()
    plt.savefig(repo_root / '40eridani_feasibility_varied.png', dpi=150)
